In [29]:
import pygame
import random
import math

In [30]:
pygame.init()

FPS = 60
width, height = 600, 600
rows, cols = 4, 4

rect_height = height // rows
rect_width = width // cols


outline_colour = (187, 173, 160)
outline_thickness = 10
background_colour = (205, 192, 180)
font_colour = (119, 110, 101)


In [31]:
class Window:
    def __init__(self):
        self.window = pygame.display.set_mode((width, height))
        pygame.display.set_caption('2048')
        self.font = pygame.font.SysFont('comicsans', 60, bold=True)
        self.move_vel = 20

    def mainloop(self):
        clock = pygame.time.Clock()
        run = True
        
        tiles = {"00":Tile(4, 0, 0), "20":Tile(4, 2, 0)}

        while run:
            clock.tick(FPS)

            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    run = False
                    break
            self.draw(tiles)
            
        pygame.quit()
    
    def draw(self, tiles):
        self.window.fill(background_colour)

        for tile in tiles.values():
            tile.draw(self.window)
        self.draw_grid()
        pygame.display.update()
        

    def draw_grid(self):
        for row in range(1, rows):
            y = row*rect_height
            pygame.draw.line(self.window, outline_colour, (0,y), (width,y), outline_thickness)
        for col in range(1,cols):
            x = col*rect_width
            pygame.draw.line(self.window, outline_colour, (x,0), (x, height), outline_thickness)
        pygame.draw.rect(self.window, outline_colour, (0, 0, width, height), outline_thickness)

class Tile:
    colours = [
        (237, 229, 218),
        (238, 225, 201),
        (243, 178, 122),
        (246, 150, 101),
        (247, 124, 95),
        (247, 95, 59),
        (237, 208, 115),
        (237, 204, 99),
        (236, 202, 80)
    ]
        
    def __init__(self, value, row, col):
        self.font = pygame.font.SysFont('comicsans', 60, bold=True)
        self.value = value
        self.row = row
        self.col = col  
        self.x = col*rect_width
        self.y = row*rect_height

    def get_colour(self):
        colour_index = int(math.log2(self.value)) - 1
        colour = self.colours[colour_index]
        return colour

    def draw(self, window):
        colour = self.get_colour()
        pygame.draw.rect(window, colour, (self.x, self.y, rect_width, rect_height))
        text = self.font.render(str(self.value), 1, font_colour)
        window.blit(text, 
                    (self.x + (rect_width - text.get_width())//2,
                    self.y + (rect_height - text.get_height())//2))
    
    def move(self, delta):
        pass

    def set_pos(self):
        pass


In [32]:
app = Window()
if __name__ == "__main__":
    app.mainloop()